<a href="https://colab.research.google.com/github/brunombo/Python/blob/master/long_TTS_xtts_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
text_to_speech_to_synthetise= "Ce document présente les Manifold-Constrained Hyper-Connections (mHC), une architecture novatrice conçue par DeepSeek-AI pour stabiliser l'entraînement des grands modèles de langage. Bien que les Hyper-Connections (HC) classiques améliorent les performances en élargissant le flux résiduel, leur nature non contrainte provoque souvent une instabilité numérique et des problèmes de divergence du signal. Pour remédier à cela, les auteurs utilisent l'algorithme de Sinkhorn-Knopp afin de projeter les connexions sur une variété de matrices doublement stochastiques, préservant ainsi la propriété de mappage d'identité. Cette approche garantit une propagation saine du signal tout en optimisant l'efficacité matérielle grâce à la fusion de noyaux et à des stratégies de mémorisation sélective. Les résultats expérimentaux démontrent que mHC surpasse les méthodes existantes en termes de scalabilité et de capacités de raisonnement sur divers tests de référence. En intégrant ces contraintes géométriques rigoureuses, le cadre mHC offre une solution robuste pour l'évolution des architectures neuronales à grande échelle."

voice_gender = 'female_fr'
# ['female_fr', 'male_fr']

In [ ]:
# Installation des dépendances
!pip install -q scipy noisereduce
!pip install -q numpy==2.0.2

# Installation du fork maintenu (supporte Python 3.12+)
!pip install -q coqui-tts
!pip install -q torchcodec

In [ ]:
! pip install torchcodec

In [ ]:
# -*- coding: utf-8 -*-
"""
TTS XTTS v2 - Version Long Audio (> 1 heure)
=============================================

Module de synthèse vocale haute qualité utilisant Coqui XTTS v2.
Optimisé pour la génération d'audio longs avec:
- enable_text_splitting=True pour découpage automatique
- Chunking intelligent par paragraphes pour textes très longs
- Concaténation audio avec crossfade
- Barre de progression et estimation temps restant
- Gestion mémoire optimisée
- Correction du bug d'argument 'language' sur l'API synthesizer

Auteur: Bruno
Date: Janvier 2025
Correction: Gemini
"""

# ==============================================================================
# IMPORTS
# ==============================================================================

from __future__ import annotations

import os
import re
import gc
import wave
import time
import hashlib
import warnings
from pathlib import Path
from typing import Optional, Union, List, Callable
from dataclasses import dataclass
from enum import Enum

import numpy as np

warnings.filterwarnings("ignore", category=UserWarning)

# ==============================================================================
# INSTALLATION (Colab)
# ==============================================================================

def install_dependencies():
    """Installe les dépendances si nécessaire (Colab)."""
    import subprocess
    import sys

    packages = [
        ("scipy", "scipy"),
        ("noisereduce", "noisereduce"),
        ("TTS", "coqui-tts"),
    ]

    for module, package in packages:
        try:
            __import__(module)
        except ImportError:
            print(f"📦 Installation de {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

    # numpy compatible
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numpy==2.0.2"])
    except:
        pass

# ==============================================================================
# CONFIGURATION
# ==============================================================================

@dataclass
class TTSConfig:
    """Configuration globale du module TTS."""
    MODEL_NAME: str = "tts_models/multilingual/multi-dataset/xtts_v2"
    SAMPLE_RATE: int = 24000
    DEFAULT_LANGUAGE: str = "fr"
    GDRIVE_FOLDER: str = "/content/drive/MyDrive/TTS_Output"

    # Configuration pour audio longs
    MAX_CHARS_PER_CHUNK: int = 500  # Caractères max par chunk pour textes très longs
    CROSSFADE_DURATION: float = 0.05  # Durée du crossfade en secondes
    ENABLE_TEXT_SPLITTING: bool = True  # Activer le split natif XTTS

    PRESET_VOICES: dict = None

    def __post_init__(self):
        self.PRESET_VOICES = {
            "female_fr": "https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/female.wav",
            "male_fr": "https://huggingface.co/spaces/coqui/xtts/resolve/main/examples/male.wav",
        }

Config = TTSConfig()

# ==============================================================================
# DEVICE MANAGEMENT
# ==============================================================================

_device = None
_device_name = "cpu"

def detect_device():
    """Détecte le meilleur device disponible."""
    global _device, _device_name
    import torch

    # Essayer TPU
    try:
        import torch_xla.core.xla_model as xm
        _device = xm.xla_device()
        _device_name = "tpu"
        print(f"⚙️ Device: TPU")
        return
    except:
        pass

    # Essayer CUDA
    if torch.cuda.is_available():
        _device = torch.device("cuda")
        _device_name = f"cuda ({torch.cuda.get_device_name(0)})"
        print(f"⚙️ Device: {_device_name}")
        return

    # Fallback CPU
    _device = torch.device("cpu")
    _device_name = "cpu"
    print(f"⚙️ Device: CPU")

# ==============================================================================
# TEXT SPLITTING UTILITIES
# ==============================================================================

class TextSplitter:
    """
    Utilitaire pour découper intelligemment les textes longs.
    Préserve la cohérence des phrases et paragraphes.
    """

    @staticmethod
    def estimate_audio_duration(text: str, chars_per_second: float = 15.0) -> float:
        """
        Estime la durée audio pour un texte donné.
        """
        return len(text) / chars_per_second

    @staticmethod
    def split_into_sentences(text: str) -> List[str]:
        """Découpe le texte en phrases."""
        # Pattern pour fin de phrase
        pattern = r'(?<=[.!?])\s+'
        sentences = re.split(pattern, text)
        return [s.strip() for s in sentences if s.strip()]

    @staticmethod
    def split_into_paragraphs(text: str) -> List[str]:
        """Découpe le texte en paragraphes."""
        paragraphs = re.split(r'\n\s*\n', text)
        return [p.strip() for p in paragraphs if p.strip()]

    @classmethod
    def split_for_long_audio(
        cls,
        text: str,
        max_chars: int = 500,
        preserve_sentences: bool = True
    ) -> List[str]:
        """
        Découpe un texte long en chunks optimaux pour la synthèse.
        """
        # Si texte court, retourner tel quel
        if len(text) <= max_chars:
            return [text]

        chunks = []

        if preserve_sentences:
            sentences = cls.split_into_sentences(text)
            current_chunk = ""

            for sentence in sentences:
                # Si la phrase seule dépasse max_chars, la découper
                if len(sentence) > max_chars:
                    if current_chunk:
                        chunks.append(current_chunk.strip())
                        current_chunk = ""
                    # Découper la phrase longue par mots
                    words = sentence.split()
                    sub_chunk = ""
                    for word in words:
                        if len(sub_chunk) + len(word) + 1 <= max_chars:
                            sub_chunk += " " + word if sub_chunk else word
                        else:
                            if sub_chunk:
                                chunks.append(sub_chunk.strip())
                            sub_chunk = word
                    if sub_chunk:
                        current_chunk = sub_chunk
                elif len(current_chunk) + len(sentence) + 1 <= max_chars:
                    current_chunk += " " + sentence if current_chunk else sentence
                else:
                    if current_chunk:
                        chunks.append(current_chunk.strip())
                    current_chunk = sentence

            if current_chunk:
                chunks.append(current_chunk.strip())
        else:
            # Découpage simple par caractères
            for i in range(0, len(text), max_chars):
                chunks.append(text[i:i + max_chars])

        return chunks


# ==============================================================================
# AUDIO PROCESSING
# ==============================================================================

class AudioProcessor:
    """Processeur audio pour post-traitement et concaténation."""

    @staticmethod
    def normalize(audio: np.ndarray, target_db: float = -3.0) -> np.ndarray:
        """Normalise l'audio au niveau cible."""
        if audio.dtype == np.int16:
            audio = audio.astype(np.float32) / 32768.0

        peak = np.max(np.abs(audio))
        if peak > 0:
            target_linear = 10 ** (target_db / 20)
            audio = audio * (target_linear / peak)

        return np.clip(audio, -1.0, 1.0)

    @staticmethod
    def crossfade(
        audio1: np.ndarray,
        audio2: np.ndarray,
        sample_rate: int,
        duration: float = 0.05
    ) -> np.ndarray:
        """
        Concatène deux segments audio avec crossfade.
        """
        # Convertir en float si nécessaire
        if audio1.dtype == np.int16:
            audio1 = audio1.astype(np.float32) / 32768.0
        if audio2.dtype == np.int16:
            audio2 = audio2.astype(np.float32) / 32768.0

        fade_samples = int(sample_rate * duration)

        # Si audio trop court pour crossfade, concaténer simplement
        if len(audio1) < fade_samples or len(audio2) < fade_samples:
            return np.concatenate([audio1, audio2])

        # Créer les courbes de fade
        fade_out = np.linspace(1.0, 0.0, fade_samples)
        fade_in = np.linspace(0.0, 1.0, fade_samples)

        # Appliquer le crossfade
        audio1_end = audio1[-fade_samples:] * fade_out
        audio2_start = audio2[:fade_samples] * fade_in

        # Assembler
        result = np.concatenate([
            audio1[:-fade_samples],
            audio1_end + audio2_start,
            audio2[fade_samples:]
        ])

        return result

    @classmethod
    def concatenate_chunks(
        cls,
        audio_chunks: List[np.ndarray],
        sample_rate: int,
        crossfade_duration: float = 0.05
    ) -> np.ndarray:
        """
        Concatène plusieurs chunks audio avec crossfade.
        """
        if not audio_chunks:
            return np.array([], dtype=np.float32)

        if len(audio_chunks) == 1:
            audio = audio_chunks[0]
            if audio.dtype == np.int16:
                audio = audio.astype(np.float32) / 32768.0
            return audio

        result = audio_chunks[0]
        if result.dtype == np.int16:
            result = result.astype(np.float32) / 32768.0

        for chunk in audio_chunks[1:]:
            result = cls.crossfade(result, chunk, sample_rate, crossfade_duration)

        return result

    @staticmethod
    def enhance(
        audio: np.ndarray,
        sample_rate: int,
        normalize: bool = True,
        warmth: bool = True
    ) -> np.ndarray:
        """Améliore la qualité audio."""
        if audio.dtype == np.int16:
            audio = audio.astype(np.float32) / 32768.0

        if warmth:
            try:
                from scipy import signal
                nyquist = sample_rate / 2
                cutoff = min(300, nyquist * 0.9) / nyquist
                b, a = signal.butter(2, cutoff, btype='low')
                bass = signal.filtfilt(b, a, audio)
                audio = audio + 0.15 * bass
            except ImportError:
                pass

        if normalize:
            peak = np.max(np.abs(audio))
            if peak > 0:
                target = 10 ** (-3.0 / 20)
                audio = audio * (target / peak)

        audio = np.clip(audio, -1.0, 1.0)
        return audio


# ==============================================================================
# PROGRESS TRACKER
# ==============================================================================

class ProgressTracker:
    """Suivi de progression avec estimation du temps restant."""

    def __init__(self, total: int, description: str = ""):
        self.total = total
        self.current = 0
        self.description = description
        self.start_time = time.time()
        self.chunk_times = []

    def update(self, chunk_duration: float = None):
        """Met à jour la progression."""
        self.current += 1
        if chunk_duration:
            self.chunk_times.append(chunk_duration)
        self._display()

    def _display(self):
        """Affiche la barre de progression."""
        elapsed = time.time() - self.start_time
        percent = (self.current / self.total) * 100

        # Estimation temps restant
        if self.chunk_times:
            avg_time = np.mean(self.chunk_times)
            remaining = avg_time * (self.total - self.current)
            eta_str = self._format_time(remaining)
        else:
            eta_str = "..."

        # Barre de progression
        bar_length = 30
        filled = int(bar_length * self.current / self.total)
        bar = "█" * filled + "░" * (bar_length - filled)

        elapsed_str = self._format_time(elapsed)

        print(f"\r{self.description} [{bar}] {self.current}/{self.total} "
              f"({percent:.1f}%) | Temps: {elapsed_str} | ETA: {eta_str}", end="")

        if self.current >= self.total:
            print()  # Nouvelle ligne à la fin

    @staticmethod
    def _format_time(seconds: float) -> str:
        """Formate un temps en secondes en HH:MM:SS."""
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)

        if hours > 0:
            return f"{hours:02d}:{minutes:02d}:{secs:02d}"
        return f"{minutes:02d}:{secs:02d}"


# ==============================================================================
# TTS ENGINE
# ==============================================================================

_tts_model = None
_voices_cache = {}
os.environ["COQUI_TOS_AGREED"] = "1"

def get_model():
    """Charge le modèle XTTS v2 avec cache."""
    global _tts_model

    if _tts_model is None:
        print("🔄 Chargement du modèle XTTS v2...")
        from TTS.api import TTS

        _tts_model = TTS(Config.MODEL_NAME)

        if _device is not None and _device_name.startswith("cuda"):
            _tts_model = _tts_model.to(_device)

        print("✓ Modèle chargé")

    return _tts_model


def get_voice_path(voice: str) -> str:
    """Obtient le chemin vers un fichier de voix."""
    global _voices_cache
    import urllib.request

    if voice in _voices_cache:
        return _voices_cache[voice]

    if os.path.isfile(voice):
        _voices_cache[voice] = voice
        return voice

    if voice in Config.PRESET_VOICES:
        url = Config.PRESET_VOICES[voice]
        path = f"/tmp/{voice}.wav"

        if not os.path.exists(path):
            print(f"📥 Téléchargement de la voix '{voice}'...")
            urllib.request.urlretrieve(url, path)

        _voices_cache[voice] = path
        return path

    raise FileNotFoundError(f"Voix '{voice}' non trouvée")


# ==============================================================================
# MAIN SYNTHESIS FUNCTIONS
# ==============================================================================

def synthesize_chunk(
    text: str,
    voice_path: str,
    language: str = "fr",
    enable_text_splitting: bool = True
) -> np.ndarray:
    """
    Synthétise un chunk de texte en audio via l'inférence directe (Low-Level).
    Bypass total du SpeakerManager pour éviter le bug FileNotFoundError .pth
    """
    model_wrapper = get_model()

    # 1. Accès "chirurgical" au modèle interne XTTS
    # C'est lui qui fait le travail, sans la couche de gestion de fichiers buggée
    if hasattr(model_wrapper, 'synthesizer'):
        xtts_model = model_wrapper.synthesizer.tts_model
    else:
        # Cas rare ou structure différente, on tente l'accès direct
        xtts_model = model_wrapper.tts_model

    # 2. Calcul manuel des latents (Empreinte vocale)
    # On transforme le fichier WAV en vecteurs mathématiques
    try:
        gpt_cond_latent, speaker_embedding = xtts_model.get_conditioning_latents(
            audio_path=[voice_path],
            gpt_cond_len=30,
            max_ref_length=60
        )
    except Exception as e:
        print(f"⚠️ Erreur calcul latents: {e}")
        raise e

    # 3. Inférence directe
    # On appelle la fonction de génération pure, sans passer par tts()
    try:
        out = xtts_model.inference(
            text=text,
            language=language,
            gpt_cond_latent=gpt_cond_latent,
            speaker_embedding=speaker_embedding,
            temperature=0.7,        # Paramètre standard pour la créativité
            length_penalty=1.0,     # Pénalité de longueur
            repetition_penalty=2.0, # Évite les bégaiements
            top_k=50,
            top_p=0.8,
            enable_text_splitting=enable_text_splitting
        )

        # Le résultat est généralement dans un dictionnaire sous la clé 'wav'
        if isinstance(out, dict) and 'wav' in out:
            wav = out['wav']
        else:
            wav = out

        # S'assurer que c'est bien un numpy array sur CPU
        if hasattr(wav, 'cpu'):
            wav = wav.cpu().numpy()
        if isinstance(wav, list):
            wav = np.array(wav, dtype=np.float32)

        return wav

    except Exception as e:
        print(f"⚠️ Erreur lors de l'inférence directe : {e}")
        raise e


def text_to_speech_long(
    text: str,
    voice: str = "female_fr",
    language: str = "fr",
    output_path: Optional[str] = None,
    enhance: bool = False,
    use_gdrive: bool = False,
    gdrive_folder: str = None,
    max_chars_per_chunk: int = None,
    show_progress: bool = True,
    enable_text_splitting: bool = True
) -> dict:
    """
    Génère un fichier audio long (> 1 heure) à partir de texte.
    """
    import torch

    # Configuration
    max_chars = max_chars_per_chunk or Config.MAX_CHARS_PER_CHUNK
    voice_path = get_voice_path(voice)

    # Estimation initiale
    estimated_duration = TextSplitter.estimate_audio_duration(text)
    print(f"\n📝 Texte: {len(text):,} caractères")
    print(f"⏱️  Durée estimée: {ProgressTracker._format_time(estimated_duration)}")

    # Découper le texte
    chunks = TextSplitter.split_for_long_audio(text, max_chars=max_chars)
    print(f"📦 Chunks: {len(chunks)}")

    # Initialiser la progression
    progress = None
    if show_progress:
        progress = ProgressTracker(len(chunks), "🎙️ Synthèse")

    # Générer l'audio chunk par chunk
    audio_chunks = []

    for i, chunk in enumerate(chunks):
        chunk_start = time.time()

        try:
            wav = synthesize_chunk(
                text=chunk,
                voice_path=voice_path,
                language=language,
                enable_text_splitting=enable_text_splitting
            )
            audio_chunks.append(wav)

        except Exception as e:
            print(f"\n⚠️ Erreur chunk {i+1}: {e}")
            # Continuer avec les autres chunks
            continue

        # Libérer la mémoire GPU périodiquement
        if _device_name.startswith("cuda") and (i + 1) % 10 == 0:
            torch.cuda.empty_cache()

        chunk_duration = time.time() - chunk_start
        if progress:
            progress.update(chunk_duration)

    if not audio_chunks:
        raise RuntimeError("Aucun audio généré")

    print("\n🔗 Concaténation des chunks...")

    # Concaténer avec crossfade
    final_audio = AudioProcessor.concatenate_chunks(
        audio_chunks,
        Config.SAMPLE_RATE,
        Config.CROSSFADE_DURATION
    )

    # Libérer les chunks de la mémoire
    del audio_chunks
    gc.collect()
    if _device_name.startswith("cuda"):
        torch.cuda.empty_cache()

    # Post-traitement
    if enhance:
        print("✨ Post-traitement...")
        final_audio = AudioProcessor.enhance(
            final_audio,
            Config.SAMPLE_RATE,
            normalize=True,
            warmth=True
        )
    else:
        final_audio = AudioProcessor.normalize(final_audio)

    # Convertir en int16
    final_audio = (final_audio * 32767).astype(np.int16)

    # Générer le nom de fichier
    if output_path is None:
        h = hashlib.md5(text[:100].encode()).hexdigest()[:8]
        output_path = f"tts_long_{voice}_{h}.wav"

    # Dossier de sortie
    if use_gdrive:
        folder = Path(gdrive_folder or Config.GDRIVE_FOLDER)
        folder.mkdir(parents=True, exist_ok=True)
        final_path = folder / Path(output_path).name
    else:
        final_path = Path(output_path)

    # Sauvegarder
    print(f"💾 Sauvegarde: {final_path}")
    with wave.open(str(final_path), "wb") as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(Config.SAMPLE_RATE)
        wav_file.writeframes(final_audio.tobytes())

    # Calculer la durée réelle
    duration = len(final_audio) / Config.SAMPLE_RATE

    print(f"\n✅ Audio généré avec succès!")
    print(f"   📁 Fichier: {final_path}")
    print(f"   ⏱️  Durée: {ProgressTracker._format_time(duration)}")
    print(f"   📦 Chunks: {len(chunks)}")
    print(f"   🎤 Voix: {voice}")

    return {
        'path': str(final_path),
        'sample_rate': Config.SAMPLE_RATE,
        'duration_seconds': duration,
        'duration_formatted': ProgressTracker._format_time(duration),
        'audio_data': final_audio,
        'voice': voice,
        'language': language,
        'device': _device_name,
        'chunks_count': len(chunks),
        'text_length': len(text)
    }


def text_to_speech(
    text: str,
    voice: str = "female_fr",
    language: str = "fr",
    output_path: Optional[str] = None,
    enhance: bool = False,
    use_gdrive: bool = False,
    gdrive_folder: str = None,
    enable_text_splitting: bool = True
) -> dict:
    """
    Génère un fichier audio à partir de texte avec XTTS v2.
    """
    # Basculer automatiquement vers la version long pour textes > 10000 chars
    if len(text) > 10000:
        print("📢 Texte long détecté - utilisation de text_to_speech_long()")
        return text_to_speech_long(
            text=text,
            voice=voice,
            language=language,
            output_path=output_path,
            enhance=enhance,
            use_gdrive=use_gdrive,
            gdrive_folder=gdrive_folder,
            enable_text_splitting=enable_text_splitting
        )

    voice_path = get_voice_path(voice)

    # Générer l'audio avec enable_text_splitting
    wav = synthesize_chunk(
        text=text,
        voice_path=voice_path,
        language=language,
        enable_text_splitting=enable_text_splitting
    )

    # Post-traitement
    if enhance:
        audio = AudioProcessor.enhance(wav, Config.SAMPLE_RATE)
    else:
        audio = AudioProcessor.normalize(wav)

    audio = (audio * 32767).astype(np.int16)

    # Nom de fichier
    if output_path is None:
        h = hashlib.md5(text.encode()).hexdigest()[:8]
        output_path = f"tts_{voice}_{h}.wav"

    # Dossier de sortie
    if use_gdrive:
        folder = Path(gdrive_folder or Config.GDRIVE_FOLDER)
        folder.mkdir(parents=True, exist_ok=True)
        final_path = folder / Path(output_path).name
    else:
        final_path = Path(output_path)

    # Sauvegarder
    with wave.open(str(final_path), "wb") as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(Config.SAMPLE_RATE)
        wav_file.writeframes(audio.tobytes())

    duration = len(audio) / Config.SAMPLE_RATE

    print(f"✓ Audio généré: {final_path}")
    print(f"  Durée: {duration:.2f}s | Voix: {voice}")

    return {
        'path': str(final_path),
        'sample_rate': Config.SAMPLE_RATE,
        'duration_seconds': duration,
        'audio_data': audio,
        'voice': voice,
        'language': language,
        'device': _device_name
    }


# ==============================================================================
# UTILITIES
# ==============================================================================

def preview_audio(result: dict) -> None:
    """Prévisualise l'audio dans le notebook."""
    from IPython.display import Audio, display

    audio = result['audio_data']
    if audio.dtype == np.int16:
        audio = audio.astype(np.float32) / 32768.0

    display(Audio(audio, rate=result['sample_rate']))


def list_voices() -> list:
    """Liste les voix disponibles."""
    return list(Config.PRESET_VOICES.keys())


def list_languages() -> list:
    """Liste les langues supportées."""
    return ["en", "es", "fr", "de", "it", "pt", "pl", "tr",
            "ru", "nl", "cs", "ar", "zh-cn", "ja", "hu", "ko", "hi"]


def clear_cache():
    """Libère la mémoire."""
    global _tts_model
    import torch

    _tts_model = None
    gc.collect()

    if _device_name.startswith("cuda"):
        torch.cuda.empty_cache()

    print("✓ Cache vidé")


def estimate_duration(text: str) -> dict:
    """
    Estime la durée audio pour un texte.
    """
    duration = TextSplitter.estimate_audio_duration(text)
    chunks = len(TextSplitter.split_for_long_audio(text))

    return {
        'chars': len(text),
        'estimated_seconds': duration,
        'estimated_formatted': ProgressTracker._format_time(duration),
        'chunks_estimate': chunks
    }


# ==============================================================================
# ALIASES
# ==============================================================================

tts = text_to_speech
tts_long = text_to_speech_long


# ==============================================================================
# INITIALIZATION
# ==============================================================================

def init():
    """Initialise le module."""
    detect_device()
    print("✅ Module XTTS v2 Long Audio chargé")
    print(f"   Device: {_device_name}")
    print(f"   Voix: {list_voices()}")
    print(f"   enable_text_splitting: activé par défaut")


# Auto-init
if __name__ != "__main__":
    try:
        detect_device()
    except:
        pass


# ==============================================================================
# EXAMPLE USAGE
# ==============================================================================

if __name__ == "__main__":
    # Installation si nécessaire
    install_dependencies()

    # Initialisation
    init()

    # Exemple avec texte court
    print("\n" + "="*60)
    print("EXEMPLE 1: Texte court")
    print("="*60)

    short_text = """
    Ce document présente les Manifold-Constrained Hyper-Connections,
    une architecture novatrice conçue par DeepSeek-AI pour stabiliser
    l'entraînement des grands modèles de langage.
    """

    result = text_to_speech(
        text=short_text.strip(),
        voice="female_fr",
        enhance=True
    )

    print(f"\nRésultat: {result['duration_seconds']:.2f}s")

    # Exemple avec texte long (simulé)
    print("\n" + "="*60)
    print("EXEMPLE 2: Estimation pour texte long")
    print("="*60)

    # Simuler un texte de ~1 heure (environ 54000 caractères)
    long_text = short_text.strip() * 300  # ~54000 chars ≈ 1 heure

    estimation = estimate_duration(long_text)
    print(f"\nEstimation pour {estimation['chars']:,} caractères:")
    print(f"  Durée: {estimation['estimated_formatted']}")
    print(f"  Chunks: {estimation['chunks_estimate']}")

    # Pour générer réellement:
    # result = text_to_speech_long(long_text, voice="female_fr", show_progress=True)

⚙️ Device: cuda (Tesla T4)
✅ Module XTTS v2 Long Audio chargé
   Device: cuda (Tesla T4)
   Voix: ['female_fr', 'male_fr']
   enable_text_splitting: activé par défaut

EXEMPLE 1: Texte court
🔄 Chargement du modèle XTTS v2...
✓ Modèle chargé
✓ Audio généré: tts_female_fr_151473ed.wav
  Durée: 9.79s | Voix: female_fr

Résultat: 9.79s

EXEMPLE 2: Estimation pour texte long

Estimation pour 55,200 caractères:
  Durée: 01:01:20
  Chunks: 108


In [ ]:
# Lire le fichier
with open("mon_texte_long.txt", "r", encoding="utf-8") as f:
    texte_complet = f.read()

# Lancer la génération
text_to_speech_long(
    text=texte_complet,
    voice="female_fr",
    language="fr"
)


📝 Texte: 6,715 caractères
⏱️  Durée estimée: 07:27
📦 Chunks: 16
🎙️ Synthèse [██████████████████████████████] 16/16 (100.0%) | Temps: 03:10 | ETA: 00:00

🔗 Concaténation des chunks...
💾 Sauvegarde: tts_long_female_fr_8aba435b.wav

✅ Audio généré avec succès!
   📁 Fichier: tts_long_female_fr_8aba435b.wav
   ⏱️  Durée: 06:37
   📦 Chunks: 16
   🎤 Voix: female_fr


{'path': 'tts_long_female_fr_8aba435b.wav',
 'sample_rate': 24000,
 'duration_seconds': 397.586,
 'duration_formatted': '06:37',
 'audio_data': array([28, 21, 31, ...,  3,  7,  1], dtype=int16),
 'voice': 'female_fr',
 'language': 'fr',
 'device': 'cuda (Tesla T4)',
 'chunks_count': 16,
 'text_length': 6715}